###  Pinkbook2017_3.2

In [1]:
from databaker.framework import *
import pandas as pd

Use common 'fetch_sources' notebook to get a local copy of all source data.

In [2]:
%run fetch_sources.ipynb

In [3]:
tab = loadxlstabs(pbFile, sheetids='3.2')[0]
savepreviewhtml(tab)

Loading in\pinkbook2017chapter3.xls which has size 1248256 bytes
Table names: ['3.2']


3.2,Transport,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,£ million
,,,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0
,,,,,,,,,,,,,,,,,,,,,,
,Exports (Credits),,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,
,Sea transport,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,
,Passenger,,,,,,,,,,,,,,,,,,,,,
,Passenger revenue,FJAL,688.0,462.0,454.0,622.0,488.0,558.0,993.0,810.0,557.0,407.0,357.0,394.0,546.0,791.0,1291.0,1267.0,1998.0,1751.0,1813.0,2198.0
,Time charter receipts,FJAM,9.0,-,9.0,8.0,-,11.0,-,36.0,51.0,37.0,73.0,75.0,75.0,96.0,158.0,90.0,57.0,37.0,36.0,58.0


In [4]:
observations = tab.excel_ref('D9').expand(DOWN).expand(RIGHT).is_not_blank()

In [5]:
Code = tab.excel_ref('C9').expand(DOWN).is_not_blank()

In [6]:
Year = tab.excel_ref('D3').expand(RIGHT).is_not_blank()

In [7]:
Commerce = tab.excel_ref('B').expand(DOWN).by_index([5,66,128]) - tab.excel_ref('B183').expand(DOWN)

In [8]:
Modeoftransport = tab.excel_ref('B').expand(DOWN).by_index([7,31,45,51,57,59,61,63,68,97,103,107,111,113,117,
                                                            119,121,123,125,130,134,141,148,151,152,154,164,170,176,178,180,182])
                                                                

In [9]:
Modeoftransport = Modeoftransport - tab.excel_ref('B183').expand(DOWN)

In [10]:
Services = tab.excel_ref('B').expand(DOWN).by_index([9,16,20,25,27,29,33,34,36,37,39,41,43,63,70,77,81,85,87,89,93,95,97,105,
                                                     125,130,154,162,182]) - tab.excel_ref('B183').expand(DOWN)

In [11]:
Typeoftransport = tab.excel_ref('B').expand(DOWN).is_not_blank().is_not_whitespace() -Services - Modeoftransport - tab.excel_ref('B1').expand(UP)

In [12]:
Currency = tab.excel_ref('W1')

In [57]:
Dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDim(Year,'Year', DIRECTLY,ABOVE),
            HDim(Code,'CDID',DIRECTLY,LEFT),
            HDimConst('Unit', '£ Millions'), 
            HDimConst('Measure Type','GBP Total'),            
            HDim(Typeoftransport, 'Product', DIRECTLY, LEFT),
            HDim(Services, 'Type of Services', CLOSEST, ABOVE),
            HDim(Modeoftransport, 'Mode of Transport', CLOSEST, ABOVE),
            HDim(Commerce, 'Flow', CLOSEST, ABOVE)
    
]

In [58]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)

In [59]:
savepreviewhtml(c1)

In [60]:
new_table = c1.topandas()

In [61]:
new_table = new_table.drop('DATAMARKER', axis=1)

In [62]:
new_table['TIME'] = new_table['Year'].map(lambda cell:cell.replace('.0', ''))

In [63]:
new_table = new_table.drop('Year', axis=1)

In [64]:
new_table['Mode of Transport'] = new_table['Mode of Transport'].map(lambda x: x.rstrip('12'))

In [65]:
new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)
new_table = new_table[new_table['Value'] != '']

In [66]:
new_table.rename(index= str, columns= {'TIME':'Year'}, inplace = True)

In [67]:
new_table['Product'] = new_table['Product'].str.strip()
new_table['Year'] = new_table['Year'].str.strip()
new_table['Geography'] = new_table['Geography'].str.strip()
new_table['CDID'] = new_table['CDID'].str.strip()
new_table['Flow'] = new_table['Flow'].str.strip()
new_table['Type of Services'] = new_table['Type of Services'].str.strip()
new_table['Mode of Transport'] = new_table['Mode of Transport'].str.strip()

In [68]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('Exports (Credits)', 'Exports'))
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('Imports (Debits)', 'Imports'))

In [69]:
new_table['Services'] = new_table['Type of Services'].map(str)+'-'+new_table['Mode of Transport']

In [74]:
new_table['Services'] = new_table['Services'].map(lambda cell:cell.replace('Total-Total', 'Total'))

In [79]:
new_table.head(4)

,Geography,Year,CDID,Product,Flow,Services,Measure Type,Value,Unit
0,K02000001,1997,FJAL,Passenger revenue,Exports,Passenger-Sea transport,GBP Total,688.0,£ Millions
1,K02000001,1998,FJAL,Passenger revenue,Exports,Passenger-Sea transport,GBP Total,462.0,£ Millions
2,K02000001,1999,FJAL,Passenger revenue,Exports,Passenger-Sea transport,GBP Total,454.0,£ Millions
3,K02000001,2000,FJAL,Passenger revenue,Exports,Passenger-Sea transport,GBP Total,622.0,£ Millions


In [76]:
new_table = new_table[['Geography','Year','CDID','Product','Flow','Services','Measure Type','Value','Unit']]

In [77]:
new_table.fillna('NA', inplace = True)

In [78]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True)

pbCSV = destinationFolder / (Path(pbFilename).stem + '_3.2.csv')

new_table.to_csv(pbCSV, index = False)